In [219]:
import numpy as np
from scipy import stats, random
import pylab as plt
from itertools import compress
from tqdm import tqdm
import zarr
import numcodecs

import pylab as plt

In [323]:
###### Datastore with N samples from normal distribution

class DataStore:
    def __init__(self, filename):
        # Open (new) datastore
        self.store = zarr.DirectoryStore(filename)
        self.root = zarr.group(store = self.store)
        
        if 'samples' not in self.root.keys():
            print("Creating new datastore:", filename)
            print("Run `init` to set up storage parameters.")
            return
        
        self.x = self.root['samples/x']
        self.z = self.root['samples/z']
        self.m = self.root['metadata/needs_sim']
        self.u = self.root['metadata/intensity']
        
    def init(self, xdim, zdim):
        """Initialize data store."""
        if 'samples' in self.root.keys():
            print("Datastore is already initialized.")
            return
        self.x = self.root.zeros('samples/x', shape=(0,)+xdim, chunks=(1,)+xdim, dtype='f4')
        self.z = self.root.zeros('samples/z', shape=(0,)+(zdim,), chunks=(100,)+(zdim,), dtype='f4')
        self.m = self.root.zeros('metadata/needs_sim', shape=(0,1), chunks=(100,)+(1,), dtype='bool')
        self.u = self.root.create('metadata/intensity', shape=(0,), dtype=object, object_codec=numcodecs.Pickle())
        
    def _append_z(self, z):
        """Append z to datastore content and new slots for x."""
        # Add simulation slots
        xshape = list(self.x.shape)
        xshape[0] += len(z)
        self.x.resize(*xshape)
        
        # Add z samples
        self.z.append(z)
        
        # Register as missing
        m = np.ones((len(z),1), dtype='bool')
        self.m.append(m)
            
# General
        
    def __len__(self):
        """Returns number of samples in the datastore."""
        return len(self.z)
        
    def intensity(self, z):
        """Replace DS intensity function with max of intensity functions."""
        return max([0,]+[self.u[i](z) for i in range(len(self.u))])
        
    def _grow(self, p):
        """Grow number of samples in datastore."""
        # Proposed new samples z from p
        z_prop = p.sample()
        
        # Rejection sampling from proposal list
        accepted = []
        for z in tqdm(z_prop, desc = "Adding samples."):
            rej_prob = np.minimum(1, self.intensity(z)/p(z))
            w = np.random.rand(1)[0]
            accepted.append(rej_prob < w)
        z_accepted = z_prop[accepted, :]
        print("Adding %i new samples."%len(z_accepted))
        
        # Add new entries to datastore and update intensity function
        self._append_z(z_accepted)
        if len(z_accepted) > 0:
            self.u.resize(len(self.u)+1)
            self.u[-1] = p
            return True
        else:
            return False
        
    def sample(self, p):
        grown = self._grow(p)
        if grown:
            print("Need to run simulator!")
            return None
        
        accepted = []
        for i, z in tqdm(enumerate(self.z), desc = "Extracting samples."):
            accept_prob  = p(z)/self.intensity(z)
            if accept_prob > 1.:
                print("Warning: Run grow!")
                return None, None
            w = np.random.rand(1)[0]
            if accept_prob > w:
                accepted.append(i)
        return accepted
    
    def __getitem__(self, i):
        return self.x[i], self.z[i]
                
#    def sample(self, mu, p):
#        indices = self._sample_indices(mu, p)
#        x_sub = list(self.x[i] for 
#        z_sub = self.z[accepted, :]
#        return x_sub, z_sub
    
    def needs_sim(self):
        indices = []
        for i in range(len(self.z)):
            if self.m[i]:
                indices.append(i)
        return indices
    
    def add_sim(self, i, x):
        self.x[i] = x
        self.m[i] = False

In [324]:
class Intensity:
    def __init__(self, mu, z0, z1):
        self.mu = mu
        self.z0 = np.array(z0)
        self.z1 = np.array(z1)
        
    def sample(self):
        N = np.random.poisson(self.mu, 1)[0]
        q = np.random.rand(N, len(self.z0))
        q *= self.z1 - self.z0
        q += self.z0
        return q
    
    def __call__(self, z):
        return self._pdf(z)*self.mu

    def _pdf(self, z):
        if any(z < self.z0) or any(z > self.z1):
            return 0.
        else:
            return 1./(self.z1 - self.z0).prod()

In [341]:
ds = DataStore("test3.zarr")
ds.init(zdim = 2, xdim = (100, 100))

Datastore is already initialized.


In [344]:
p0 = Intensity(500, [0.0, 0.5], [1.0, 1.0])
indices = ds.sample(p0)

Adding samples.: 100%|██████████| 503/503 [00:00<00:00, 1723.41it/s]

Adding 122 new samples.
Need to run simulator!


In [336]:
sims = ds.needs_sim()
for i in sims:
    print(i)
    x, z = ds[i]
    ds.add_sim(i, x)

In [321]:
p0 = Intensity(100, [0.0, 0.2], [0.5, 1.0])
ds.sample(p0)

Extracting samples.: 4it [00:00, 616.42it/s]

(None, None)